# How to read this document

This document is designed to showcase the different basic functionality of Hyperspy. For this practical you're intended to build your own notebook with the help of this document. 

We've given you a spectrum image (spim) called "spim_low_loss.hspy" and its associated ADF image : "ADF_low_loss.hspy". In this practical you are supposed to explore the data and understand what they represent. To guide you, here are a few questions :

- What is the scale of the observed object ? In what energy range, the EELS was measured ?
- What are the required pre-processing steps before starting a deeper analysis of the spim ?
- Can you tell whether the spectra change spatially ? If yes, how ?
- What is the energy of these different spectral features ?

To succeed at this practical, answer those questions in a separate ipython notebook. If you want to go further, you are welcome to move to the next level.

**The jupyter notebook extensions are installed. Thus, you can navigate between sections using the left panel**

Not everything is intended to be read the from the first time. Optional comments along the document are marked by emojis: 
- ⭐ This is a tip, describing more in depth the inner workings of hyperspy.
- 🏠 It's an optional comment. Something that you may not use here but that you can use back to your lab.


# Introduction to Hyperspy

Hyperspy is a tool for the visualisation and manipulation of high-dimensional data. The main features of Hyperspy are the following:
- The Signal1D and Signal2D objects
- Domain-specific features for EELS, EDX, etc ...
- Model fitting
- Machine learning

This document will guide through the following steps : 
- Manipulation and visualisation of data using the Signal objects. 
- Various Data preprocessing such as energy alignement or deconvolution.
- How to map the different features of the spim.
- How to perform a 1D fit.

A large part of this practical is inspired from the hyperspy documentation : http://hyperspy.org/hyperspy-doc/current/index.html

# 🏠 Installing Hyperspy with anaconda

This section is intended to help you to install hyperspy on your own computer in your own lab. **You can skip this part for the practical.**

Anaconda is python package manager dedicated to science. Using Anaconda it is possible to create virtual environments. You can then manage different virtual environments to isolate different distributions of python packages. This way you can keep stable python distributions safe from unstable ones.

We describe here the few steps required to create an environment with Hyperspy.

### 🏠 Install Anaconda

Depending on your operating system install anaconda : https://www.anaconda.com/products/individual

Then, launch the Anaconda Prompt, a console should appear.

### 🏠 Create a virtual environment

Run the command : ```conda create --name myenv``` (change "myenv" by any name you like.)

Your virtual environment is created but you're still in the default one called "base". You need to activate it by typing: ```conda activate myenv```

### 🏠 Installing packages

When installing a package there are two solutions: 
1. Either it is part of the already available libraries 
2. Either you can access it from anaconda forge

(1). Run the command : ```conda install package-name``` with for example: ```conda install scikit-learn```

(2). The available python packages are listed on : https://anaconda.org/ . After searching for the library you want, you can click on its name in bold. There will be a list of commands to type and execute to install the selected package. For Hyperspy it is: ```conda install -c conda-forge hyperspy```

### 🏠 installing ipython

The next few steps will enable you to run the virtual environment you created in ipython notebooks. In the Anaconda prompt run the following commands :
- ```conda install jupyter```
- ```conda install ipykernels```
- ```python -m ipykernel install --user --name myenv```

Now you can start the ipython notebook environment using: ```jupyter notebook```. You will be able to create ipython notebooks (top right corner) with the virtual environments you want. All the packages installed (and their versions) will be specific to that environment. 

You can also check https://github.com/adriente/practical_EELS_QEM to get this practical.

# Import Hyperspy

⭐ We choose first a graphical backend that manage interactivity (```%matplotlib qt```).

In python the libraries have to be imported to use their functions. For clarity, hyperspy will be imported with the name "hs". 

In [ ]:
%matplotlib qt
import hyperspy.api as hs

# Loading data

Hyperspy is able to load and manage data from many different open-source and proprietary formats, such as: 
- dm3, dm4
- emd, bcf
- tiff, jpg, etc...

For an exhaustive list : http://hyperspy.org/hyperspy-doc/current/user_guide/io.html#supported-formats

⭐ Without arguments ```hs.load()```  makes a file explorer window pop up. You can also load a list of files.

In [ ]:
spim = hs.load("spim_low_loss.hspy")
adf = hs.load("ADF_low_loss.hspy")

In [ ]:
spim.plot()

# Data structure and visualisation

The loaded data have a signal type, such as : `EELSSpectrum`, `EDSTEMSpectrum` or `EDSSEMSpectrum`.

Optionnally the data can have a title. 

The main feature of hyperspy signals is the dimension. The dimensions are either navigation or signal. For a spectrum image there are two navigation dimensions X and Y and one signal dimension Z. It is noted : `(X,Y|Z)`

⭐ If you want to developp a new application, hyperspy can support its implementation as a new signal type.

In [ ]:
print(spim)

## I. Metadata

The Signal object has a metadata attribute. Some of it is used for signal specific function (e.g. beam energy). In this tutorial all the relevant metatadata is already filled in.

🏠 You can add new metadata categories this way : `spim.metadata.Category1 = {}` and new metadata entries this way : `spim.metadata.Category1.entry1 = 'entry1 value'`

⭐For some of the proprietary file formats, some of the metadata are automatically read and loaded by hyperspy. Be careful, what is loaded and what is not.

⭐If you want to build metadata by yourself, there are some helper function such as : `spim.set_microscope_parameters()` or `spim.add_elements()`

In [ ]:
print(spim.metadata)

## I. Axes manager

The signal object has an axes_manager attribute to manage the calibrations of both the navigation axes and the signal axis. The Graphical User Interface (GUI) enables you to edit the values. 

⭐ Just typing `spim.axes_manager` prints it. You can then acces the values using e.g. : `spim.axes_manager[0].scale` or `spim.axes_manager[axis_name].size`

In [ ]:
spim.axes_manager.gui()

## I. Plotting

For 3 dimensional data, the plotting makes 2 windows pop-up. The first window contains an image in which the intensity of each pixel is the sum of the intensities of the corresponding spectrum (sum over the signal axis). A single pixel Region Of Interest is displayed in the top left of the image. The second window represents the spectrum associated to the ROI. 

You can move this ROI using arrow keys.

The adf is also plotted as an extra window.

In [ ]:
spim.plot()
adf.plot()

### I. Advanced Plotting

The code below is a way to improve the basic visualisation of hyperspy. In addition to the single pixel ROI (which cannot be removed), there is an ajustable ROI. A thrid window is also added and displays the sum of spectra in the adjustable ROI. 🏠

In [ ]:
# Get safe coordinates to initialize the ROI
half_h = spim.axes_manager[0].offset + spim.axes_manager[0].scale*0.5*spim.axes_manager[0].size 
half_v = spim.axes_manager[1].offset + spim.axes_manager[1].scale*0.5*spim.axes_manager[1].size
third_h = spim.axes_manager[0].offset + spim.axes_manager[0].scale*0.333*spim.axes_manager[0].size 
third_v = spim.axes_manager[1].offset + spim.axes_manager[1].scale*0.333*spim.axes_manager[1].size 

# Create the ROI
img_ROI=hs.roi.RectangularROI(third_h,
                              third_v,
                              half_h,
                              half_v)

# Plot the spim as usual
spim.plot()
# Link, interactively, the ROI to the spim
spim_ROI=img_ROI.interactive(spim)

# Compute interactively the sum over the ROI
spectrum=hs.interactive(spim_ROI.sum,
           event=spim_ROI.axes_manager.events.any_axis_changed)
# Plot the computed sum
spectrum.plot(True)

# Data Preprocessing

## Cropping

The crop function can be used to cut out parts of the data. **Be careful : it overwrites the data.** The syntax is as follows : `spim.crop(axis,start,stop)`. Note that axis is either an integer or an axis name. 

For `start` and `stop`, the options are :
- Integer indices : They do not take into account the calibration and correspond to matrix indices.
- Float indices : They take into account the calibration.

⭐Using `.inav` for navigation axes or `.isig` for signal axes is a more powerful way to slice data. It supports integer indexing, float indices and even string indices with e.g. `spim.inav["10 nm","20 nm"]`. It also supports advanced slicing such as `spim.isig[::2]` or `spim.inav[::,-1]`.



In [ ]:
spim.crop(0,10.0,20.0)

## Energy alignement

In low-loss, we have usually access to the zero-loss peak (zlp). It conviniently gives an accurate 0 to the energy scale (up to some precision, which depends on the instrument). Therefore, all the spectra can be aligned in energy so that the zlp is at 0 eV.

**Be careful : It overwrites the spim object.** 

In [ ]:
spim.align_zero_loss_peak()

## Deconvolution

The electrons can undergo several inelastic scattering events during their travel inside of the sample. However we want to access the single scattering distribution which directly correspond to a physically meaningfull spectrum (as direct measure of transitions probabilty). The single scattering distribution is given by : 

$$ S_{1}(E) = I_{0} t \theta(E) * \sigma(E) $$

Where $I_{0}$ is the incoming beam intensity, $t$ is the thickness of the sample, $\theta(E)$ is the zero loss peak and $\sigma(E)$ is inelastic scattering spectrum. The zero loss peak is already convolved to the spectrum. Even in the case of single scattering, the deconvolution can improve the energy resolution [Gloter et al. (2003)](https://doi.org/10.1016/S0304-3991(03)00103-7). When the sample is too thick, the spectrum is dominated by the multiple scattering distribution and $\sigma(E)$ can not be straightforwardly obtained.

For example, the double scattering distribution is given by : 

$$ S_{2}(E) = I_{0} \frac{t^{2}}{2} \theta(E) * \sigma(E) * \sigma(E) $$

In that case, it is important to perform the deconvolution with the full low loss spectrum

### Get the low losses of the substrate

Here we get an arbitrary amount of the most isolated pixels from the plasmonic structure. Therfore we will deconvolute the spectrum image with the low loss signature of the substrate. We will get the pure spectrum of the plasmonic structure.

In [ ]:
ll = spim.inav[:3,:3].sum(axis = (0,1))

### Richardson Lucy deconvolution

-First cell : The Richardson Lucy deconvolution is usually the prefered deconvolution method in EELS [Gloter et al. (2003)](https://doi.org/10.1016/S0304-3991(03)00103-7). It takes a low loss spectrum to deconvolute the spectrum with. The number of iterations is a key parameter here, depending on your dataset you should increase it or decrease it.

- Second cell : Accept the deconvolution result.

⭐ Unlike many hyperspy functions, the deconvolution does not overwrite the initial object.

In [ ]:
deconvoluted = spim.richardson_lucy_deconvolution(ll,iterations = 20)

In [ ]:
spim = deconvoluted.copy()

# Integrating peaks and generating maps

## Span ROI

**Execute this cell before the next one.** 

🏠 You don't need to understand what happens in that cell for this practical. It aims at subclassing the SpanROI of hyperspy to add a new functionality. You're welcome to use both cells below in your lab.

In [ ]:
import numpy as np

class cst_SpanROI (hs.roi.SpanROI) : 
    def mapping (self,spim = None, out_map = None) :
        out_map.data = spim.isig[self.left:self.right].sum(axis = 2).data
        return out_map

## Mapping spectral features

Two windows will pop out :
- One window reprensenting the sum of spectra of the spectrum image and a green interactive slider on top of it.
- Another window representing interactively the sum over the slice selected in the first window. 

In [ ]:
# Get safe initial coordinates for the ROI.
half_e = spim.axes_manager[2].offset + spim.axes_manager[2].scale*0.5*spim.axes_manager[2].size
third_e = spim.axes_manager[2].offset + spim.axes_manager[2].scale*0.333*spim.axes_manager[2].size

# Sum over all the spectra of the spim
full_spectrum = spim.integrate1D(axis = (0,1))
# Initialize the ROI
spectrum_ROI = cst_SpanROI(third_e,half_e)

# Plot both the spectrum and the ROI (interactively)
full_spectrum.plot(True)
spectrum_ROI.interactive(full_spectrum)

# Initialize a Signal2D which will represent the map
out = hs.signals.Signal2D(np.zeros((spim.axes_manager[0].size,spim.axes_manager[1].size)))
# Link the spim, the map and the SpanROI interactively
map=hs.interactive(spectrum_ROI.mapping,recompute_out_event = spectrum_ROI.events.changed,spim = spim,out_map = out)
map.plot()

## Save your results

Please, do not hesitate to change the filename.

In [ ]:
map.save("myfilename.hspy")

# 🏠 Converting `.hspy` data to Digital Micrograph (DM) readable format

You first need to install this plugin in DM : http://hyperspy.org/hyperspy-doc/current/user_guide/io.html#importrpl-digital-micrograph-plugin

Then you just need to save your data in `.rpl` format using hyperspy. It will produce `.rpl` and `.raw` files. Using the DM plugin you'll be able to open the `.rpl` file and read your data in DM.

In [ ]:
spim.save("mychosenname.rpl")

# Fitting 1D Spectrum

Here we will fit the sum of spectra using gaussians. In this example, we only use one gaussian (for the zero loss peak), it is up to you to to fit the plasmonic signatures with more gaussians. **Call the supervisor of this practical for help if you are stuck**

Using the fitting procedure on spectra you picked in the dataset you will be able to precisely determine the energy of the different spectral features. 

## Initializing the model

This part is divided in two cells. 

- In the first cell we initialize the objects we require : model and gaussian component. In that case, we also disable the PowerLaw background. We are mostly interested in a limited spectral range. For that we use : `model.set_signal_range`. 

- In the second cell, a window will pop up with in red the data and in blue the model (a flat line at the beginning). You can then tune the initial parameters of the gaussian using the interface below the second. It is all interactive.

In [ ]:
# Creating the model on the sum of spectra
model = spim.sum(axis = (0,1)).create_model()

# Initialize a gaussian component and adds it to the model
g = hs.model.components1D.Gaussian()
model.extend([g])

# We remove the useless background component
model.disable_background()
model.set_signal_range(0.2,5.0)

In [ ]:
model.enable_adjust_position()
model.gui()

## Fitting

In [ ]:
model.fit()

## Display the results

### Plot

Setting the keyword argument `plot_components=True` you can visualize the gaussian you added in green 

In [ ]:
model.plot(plot_components=True)

### Model parameters

A more detailed summary of the parameters obtained using fitting.

In [ ]:
model.print_current_values()